# Assignmnet 2 (100 points)

**Name:** <br>
**Email:** <br>
**Group:** A/B <br>
**Hours spend *(optional)* :** <br>

### SMS Spam Detection *(60 points)*

<p>You are hired as an AI expert in the development department of a telecommunications company. The first thing on your orientation plan is a small project that your boss has assigned you for the following given situation. Your supervisor has given away his private cell phone number on too many websites and is now complaining about daily spam SMS. Therefore, it is your job to write a spam detector in Python. </p>

<p>In doing so, you need to use a Naive Bayes classifier that can handle both bag-of-words (BoW) and tf-idf features as input. For the evaluation of your spam detector, an SMS collection is available as a dataset - this has yet to be suitably split into train and test data. To keep the costs as low as possible and to avoid problems with copyrights, your boss insists on a new development with Python.</p>

<p>Include a short description of the data preprocessing steps, method, experiment design, hyper-parameters, and evaluation metric. Also, document your findings, drawbacks, and potential improvements.</p>

<p>Note: You need to implement the bag-of-words (BoW) and tf-idf feature extractor from scratch. You can use existing python libraries for other tasks.</p>

**Dataset and Resources**

* SMS Spam Collection Dataset: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

In [2]:
import pandas as pd
import re
import math
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Step 1: Load and preprocess data
df = pd.read_csv(r"sms_spam_collection\SMSSpamCollection.csv", sep='\t', header=None, names=['label', 'message'])



def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text.split()

df['tokens'] = df['message'].apply(preprocess)

# Step 2: Bag of Words implementation
def bag_of_words(docs):
    vocabulary = {}
    bow_vectors = []
    idx = 0
    for tokens in docs:
        for token in tokens:
            if token not in vocabulary:
                vocabulary[token] = idx
                idx += 1
    for tokens in docs:
        vec = [0] * len(vocabulary)
        for token in tokens:
            vec[vocabulary[token]] += 1
        bow_vectors.append(vec)
    return bow_vectors, vocabulary

# Step 3: TF-IDF implementation
def tf_idf(docs, vocabulary=None, df_count=None, existing_vocab=False):
    N = len(docs)
    if vocabulary is None:
        vocabulary = {}
    if df_count is None:
        df_count = defaultdict(int)

    tfidf_vectors = []
    idx = len(vocabulary)

    # Count doc frequency only if not reusing vocab
    if not existing_vocab:
        for tokens in docs:
            for token in set(tokens):
                df_count[token] += 1
                if token not in vocabulary:
                    vocabulary[token] = idx
                    idx += 1

    for tokens in docs:
        vec = [0] * len(vocabulary)
        token_counts = defaultdict(int)
        for token in tokens:
            token_counts[token] += 1
        for token, count in token_counts.items():
            if token in vocabulary:
                tf = count / len(tokens)
                idf = math.log(N / (df_count[token] + 1))
                vec[vocabulary[token]] = tf * idf
        tfidf_vectors.append(vec)
    return tfidf_vectors, vocabulary, df_count

# Step 4: Naive Bayes Classifier
class NaiveBayesClassifier:
    def fit(self, X, y):
        self.classes = list(set(y))
        self.class_probs = {}
        self.word_probs = {}
        for c in self.classes:
            X_c = [X[i] for i in range(len(y)) if y[i] == c]
            total_words = np.sum(X_c, axis=0)
            class_total = np.sum(total_words)
            self.class_probs[c] = len(X_c) / len(y)
            self.word_probs[c] = (np.array(total_words) + 1) / (class_total + len(total_words))

    def predict(self, X):
        preds = []
        for x in X:
            class_scores = {}
            for c in self.classes:
                log_prob = np.log(self.class_probs[c])
                log_prob += np.sum(x * np.log(self.word_probs[c]))
                class_scores[c] = log_prob
            preds.append(max(class_scores, key=class_scores.get))
        return preds

# Step 5: Run Experiment
def run_experiment(feature_type='bow'):
    X_train, X_test, y_train, y_test = train_test_split(df['tokens'], df['label'], test_size=0.2, random_state=42)

    if feature_type == 'bow':
        X_train_vec, vocab = bag_of_words(X_train)
        X_test_vec = [[0] * len(vocab) for _ in X_test]
        for i, tokens in enumerate(X_test):
            for token in tokens:
                if token in vocab:
                    X_test_vec[i][vocab[token]] += 1

    elif feature_type == 'tfidf':
        # Train TF-IDF with new vocab
        X_train_vec, vocab, df_count = tf_idf(X_train)

        # Reuse vocab and doc freq for test
        X_test_vec, _, _ = tf_idf(X_test, vocabulary=vocab, df_count=df_count, existing_vocab=True)

    model = NaiveBayesClassifier()
    model.fit(X_train_vec, list(y_train))
    preds = model.predict(X_test_vec)

    print(f"\nResults using {feature_type.upper()}:")
    print(classification_report(y_test, preds))



# Run both experiments
run_experiment('bow')
run_experiment('tfidf')


# 📄 **SMS Spam Detection: Summary Report**
#
# **Data Preprocessing:**
# - **Lowercasing**: All text is converted to lowercase.
# - **Noise Removal**: Non-alphabetic characters (punctuation, digits, etc.) are removed.
# - **Tokenization**: The message is split into a list of words (tokens).
#
# **Methodology:**
# - Two feature extraction methods: **Bag of Words (BoW)** and **TF-IDF (Term Frequency-Inverse Document Frequency)**.
# - A custom **Naive Bayes Classifier** is used to classify the messages as spam or ham.
#
# **Experiment Design:**
# - **Train/Test Split**: 80% training, 20% testing.
# - **Features Used**: Bag of Words and TF-IDF.
# - **Workflow**:
#   1. Preprocess the messages.
#   2. Vectorize the tokens using BoW or TF-IDF.
#   3. Train the classifier on the training data.
#   4. Evaluate performance on test data.
#
# **Hyperparameters:**
# - **Laplace Smoothing (α)**: Implicitly applied by adding 1 to word counts.
# - **TF-IDF IDF Smoothing**: Added +1 in the denominator to avoid division by zero.
#
# **Evaluation Metric**:
# - **Classification Report**: Includes accuracy, precision, recall, and F1-score for both classes (ham and spam).
#
# **Findings:**
# - **BoW** and **TF-IDF** both achieved high accuracy (~97%).
# - **BoW** slightly outperforms **TF-IDF** in this case due to the small size of the dataset.
#
# **Drawbacks:**
# - **No Stopwords Removal**: Common words like "the", "is" may dilute the signal.
# - **Limited Tokenization**: Misses advanced tokenization (e.g., contractions, negations).
# - **Vocabulary Size Mismatch**: The vocab size differs between BoW and TF-IDF, leading to shape mismatches.
# - **Naive Bayes Limitation**: Assumes word independence, which is not always true.
#
# **Potential Improvements:**
# - Use **TfidfVectorizer** or **CountVectorizer** from sklearn for better preprocessing and vectorization handling.
# - Add **stopword removal** and **lemmatization**.
# - Test more advanced models (e.g., logistic regression, decision trees).
# - Use **N-grams** for better contextual understanding of words.


Results using BOW:
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       966
        spam       1.00      0.92      0.96       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115


Results using TFIDF:
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       1.00      0.72      0.84       149

    accuracy                           0.96      1115
   macro avg       0.98      0.86      0.91      1115
weighted avg       0.96      0.96      0.96      1115



 ### Search Engine *(40 points)*

Your boss is impressed with your spam detector and assigns you a new task. As part of improving internal tools, the company wants a search engine that can search through SMS messages and rank them by relevance. Implement the PageRank algorithm from scratch to score each SMS message based on its importance in the document graph.

*   Compute TF-IDF vectors for all SMS messages (you can leverage previous implementation)
*   Construct a document graph, where each node represents an SMS message and edges are the links between nodes.
*  Implement the PageRank algorithm from scratch to assign an importance score to each SMS message based on its position in the document graph.

#### Hint : You can use the previous dataset or any dataset from your choice.




## You might need the follwoing formulas for your implementation

---

### 1) Cosine Similarity Between Two Document Vectors

Cosine similarity measures how similar two vectors are based on the angle between them:

$$
\text{cosine\_sim}(A, B) = \frac{A \cdot B}{\|A\| \cdot \|B\|}
$$

- \( A \cdot B \): Dot product of vectors \( A \) and \( B \)  
- \( \|A\| \): Euclidean norm (magnitude) of vector \( A \)  
- \( \|B\| \): Euclidean norm of vector \( B \)

**Use case**: Comparing TF-IDF vectors to measure similarity between two messages.

---

### 2) PageRank of a Node \( i \)

PageRank estimates the importance of a document based on its connections in a graph:

$$
PR(i) = \frac{1 - d}{N} + d \sum_{j \in M(i)} \frac{PR(j)}{L(j)}
$$

Where:
- \( PR(i) \): PageRank score of node \( i \)  
- \( d \): Damping factor (typically 0.85)  
- \( N \): Total number of nodes (documents) in the graph  
- \( M(i) \): Set of nodes that link to node \( i \)  
- \( L(j) \): Number of outbound links from node \( j \)  

**Interpretation**:  
- A document is important if **important documents link to it**.  
- The score is split among a node’s outbound links.  
- The **teleportation term** $\text(\frac{1 - d}{N})$ accounts for random jumps, ensuring stability and fairness.

---



In [4]:
import pandas as pd
import numpy as np
import re
from math import log
from collections import defaultdict
from nltk.corpus import stopwords


# Load and preprocess data
def preprocess(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    tokens = [word for word in text.split() if word not in stop_words]
    return tokens

def compute_tf_idf(docs):
    tf_idf_vectors = []
    N = len(docs)
    df = defaultdict(int)

    for doc in docs:
        unique_terms = set(doc)
        for term in unique_terms:
            df[term] += 1

    for doc in docs:
        tf_idf = {}
        term_counts = defaultdict(int)
        for term in doc:
            term_counts[term] += 1
        doc_len = len(doc)
        for term, count in term_counts.items():
            tf = count / doc_len
            idf = log(N / (1 + df[term]))
            tf_idf[term] = tf * idf
        tf_idf_vectors.append(tf_idf)

    return tf_idf_vectors

def cosine_similarity(vec1, vec2):
    common = set(vec1.keys()) & set(vec2.keys())
    dot_product = sum(vec1[x] * vec2[x] for x in common)
    norm1 = np.sqrt(sum(v**2 for v in vec1.values()))
    norm2 = np.sqrt(sum(v**2 for v in vec2.values()))
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

def build_graph(tf_idf_vectors, threshold=0.2):
    N = len(tf_idf_vectors)
    graph = [[] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            if i != j:
                sim = cosine_similarity(tf_idf_vectors[i], tf_idf_vectors[j])
                if sim > threshold:
                    graph[i].append(j)
    return graph

def page_rank(graph, d=0.85, max_iter=100, tol=1e-6):
    N = len(graph)
    pr = [1/N] * N

    for _ in range(max_iter):
        new_pr = [ (1 - d) / N ] * N
        for i in range(N):
            if graph[i]:
                for j in graph[i]:
                    new_pr[j] += d * (pr[i] / len(graph[i]))
        if max(abs(new_pr[i] - pr[i]) for i in range(N)) < tol:
            break
        pr = new_pr
    return pr

def main():
    df = pd.read_csv(r"sms_spam_collection\SMSSpamCollection.csv", sep='\t', header=None, names=['label', 'message'])
    df['tokens'] = df['message'].apply(preprocess)
    tf_idf_vectors = compute_tf_idf(df['tokens'])
    graph = build_graph(tf_idf_vectors)
    scores = page_rank(graph)
    df['pagerank'] = scores
    top = df.sort_values('pagerank', ascending=False).head(10)
    print("Top 10 Important Messages:\n")
    print(top[['pagerank', 'message']])

if __name__ == "__main__":
    import nltk
    nltk.download('stopwords')
    main()

[nltk_data] Downloading package stopwords to C:\Users\FATIMA
[nltk_data]     BAIG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Top 10 Important Messages:

      pagerank                          message
4856  0.001483                     Same to u...
1553  0.001483                         U too...
5173  0.001207                             U 2.
1394  0.001150                          Oh ok..
1834  0.001132         When should I come over?
502   0.001132             When can ü come out?
2621  0.001132                        How come?
1504  0.001130  Ill be there on  &lt;#&gt;  ok.
2537  0.001069         You do what all you like
1273  0.001056                            Ok...


### Additional Experiments *(5 additional points - <span style="color: red;">Optional</span>)*